In [ ]:
from http.server import HTTPServer, SimpleHTTPRequestHandler

httpd = HTTPServer(('localhost', 8000), SimpleHTTPRequestHandler)
httpd.serve_forever()

In [ ]:
!pip install pyftpdlib

In [ ]:
from pyftpdlib import servers
from pyftpdlib.handlers import FTPHandler
address = ("", 21)  # listen on every IP on my machine on port 21
server = servers.FTPServer(address, FTPHandler)
server.serve_forever()

In [ ]:
import os
import sys
from hashlib import md5

from pyftpdlib.handlers import FTPHandler
from pyftpdlib.servers import FTPServer
from pyftpdlib.authorizers import DummyAuthorizer, AuthenticationFailed


class DummyMD5Authorizer(DummyAuthorizer):

    def validate_authentication(self, username, password, handler):
        if sys.version_info >= (3, 0):
            password = md5(password.encode('latin1'))
        hash = md5(password).hexdigest()
        try:
            if self.user_table[username]['pwd'] != hash:
                raise KeyError
        except KeyError:
            raise AuthenticationFailed


def main():
    # get a hash digest from a clear-text password
    hash = md5('12345').hexdigest()
    authorizer = DummyMD5Authorizer()
    authorizer.add_user('user', hash, os.getcwd(), perm='elradfmwMT')
    authorizer.add_anonymous(os.getcwd())
    handler = FTPHandler
    handler.authorizer = authorizer
    server = FTPServer(('', 2121), handler)
    server.serve_forever()

main()

In [ ]:
import shutil

shutil.copy('utils.py', 'http://localhost:8000/')

In [ ]:
import bcrypt
from basicauth import encode
import json

resource_file = "ftpServer/passwords.json"
def encode(username,password):
    return (bcrypt.hashpw((username+password).encode('utf-8'), bcrypt.gensalt())).decode('utf-8')

def add_user(username,password):
    with open("ftpServer/passwords.json", 'r') as openfile:
        passw  = json.load(openfile)

    if username in passw.keys():
        print("user already exists")
        return
    else:
        passw[username] = str(encode(username,password))
        with open("ftpServer/passwords.json", "w") as outfile:
            json.dump(passw, outfile)
        return username

def check_login(username,password):
    with open("ftpServer/passwords.json", 'r') as openfile:
        passw  = json.load(openfile)
    if username in passw.keys():
        check =(username+password).encode('utf-8')
        test = passw[username].encode('utf-8')
        print("tes")
        print(test)
        if bcrypt.checkpw(check, test):

            return username
    return

def create_username():
     try: 
         username = add_user(input("enter username:"),input("enter password:"))
         print("Added User! %s"%username)
     except Exception as e:
         print("Failed to add user %s! ... user already exists??"%username)

def login():
     if check_login(input("enter username:"),input("enter password:")):
        print("Login Success!!")
     else:
        print("there was a problem logging in")

# while True:
#     {'c':create_username,'l':login}.get(input("(c)reate user\n(l)ogin\n------------\n>").lower(),login)()
# create_username()
login()

In [ ]:
import requests

test_file = open("utils.py", "rb")
test_url = "http://localhost:8000/"
test_response = requests.post(test_url, files = {"form_field_name": test_file})

In [ ]:
import socket
import threading
import hashlib
import os
import http.server
import time
import json
import base64
import shutil
import sys



peersock = []           #List maintaining socket object of all clients
fix_listen_port=9578    #Node will listen for peering request on this port.
fix_fetch_port=2100     #This port is used for fetching the file from other peer node.
fix_browser_port=1500   #This port will handle HTTP request from clients.


#Global data dictionary will maintain all data and shared with peers
globaldata={'sha-1':[],'content type':[],'content len':[],'ip':[],'port':[]}

#This dictionary will store files published by node itself.
#It will be printed when SHOW_PUBLISHED is entered.
metadata_published={'sha-1':[],'content type':[],'content len':[]}

#This is local database storing Hash value and corresponding file name.
database={'sha-1':[],'filename':[]}

#This dictionary will maintain IP address and port no. of peers connected.
#It will be printed when SHOW_PEERS is entered.
peers={'NAME OF PEER':[], 'IP ADDRESS':[],'PORT':[]}

In [ ]:


#Function to check weather file exist in node's database or not.
def is_present(dict,hash):
    index=search(dict,hash)
    if index==-1:
        return "not present"
    else:
        return "present"




#Function to find Filename from Hash value.
#Filename is present in node's own database.
def filename(extracted_hash):
    index_in_database = search(database, extracted_hash)
    file = database['filename'][index_in_database]
    print(file)
    return file




#Function to print a table of meta data information currently being published by this node
#This function will respond to command : SHOW_PUBLISHED
def show_published():
    for key,value in metadata_published.items():
        print(key,value)




#Function to print a table of meta data and endpoint information stored by this node
#This function will respond to command : SHOW_METADATA
def show_metadata():
    for key,value in globaldata.items():
        print(key,value)




#Function to print a table of peer endpoints
#This function will respond to command : SHOW_PEERS
def show_peers():
    for key,value in peers.items():
        print(key,value)




#Function to update dictionary
def update(dictionary, key, value):
    if type(dictionary[key]) == list:
        dictionary[key].append(value)
    else:
        dictionary[key] = [dictionary[key],value]




#Function to find index of any value present in dictionary
def search(diction, searchFor):
    list = diction['sha-1']
    if searchFor in diction['sha-1']:
        return diction['sha-1'].index(searchFor)
    else:
        return -1




#Function to publish a file into p2pweb.
def publish(filename, dot, contenttype):
    full = filename + dot + contenttype
    try:
        fileopen = open(full,'r')
    except FileNotFoundError:
        print("File not found")
        return
    md,ct,cl=sha1(filename, dot, contenttype)       #Calling sha1 Function to calculate hash of the file.
    update(globaldata, 'sha-1', md)                 #Updating globaldata.
    update(globaldata, 'content type', ct)
    update(globaldata, 'content len', cl)
    update(globaldata, 'ip', ip_dc_machine)
    update(globaldata, 'port', fix_listen_port)
    update(metadata_published, 'sha-1', md)         #Updating local dictionary.
    update(metadata_published, 'content type', ct)
    update(metadata_published, 'content len', cl)
    update(database,'sha-1',md)                     #Updating local database.
    update(database,'filename',full)
    data=json.dumps(globaldata)
    for c in peersock:                              #Sending updated globaldata to all peers.
        if c == ip_dc_machine:
            pass
        else:
            updateSock=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            updateSock.connect((c,9578))
            updateSock.sendall(str.encode(data))
            updateSock.close()
    return




#Function to calculate hash of the file
def sha1(filename, dot, contenttype):
    full = filename + dot + contenttype
    print(full)
    if (contenttype == "png"):                          #Calculating Hash for Image
        image = open(full, 'rb')
        imstr = base64.b64encode(image.read())
        m = hashlib.sha1(imstr)
        Msg_Digest= m.hexdigest()
        Content_size= os.stat(full).st_size
        return (Msg_Digest, contenttype, Content_size)
    else:                                               #Calculating Hash for html/txt.
        txtfile = open(full, "rb")
        Read_File = txtfile.read()
        m = hashlib.sha1(Read_File)
        Msg_Digest = m.hexdigest()
        Content_size=os.path.getsize(full)
        return (Msg_Digest, contenttype, Content_size)




#Function to un-publish a file from p2pweb.
def unpublish(hash_to_unpublish):
    index=search(globaldata,hash_to_unpublish)
    del globaldata["sha-1"][index]                       #Updating global data.
    del globaldata["content type"][index]
    del globaldata["content len"][index]
    del globaldata["ip"][index]
    del globaldata["port"][index]
    index1=search(metadata_published,hash_to_unpublish)
    del metadata_published["sha-1"][index1]              #Updating local dictionary.
    del metadata_published["content type"][index1]
    del metadata_published["content len"][index1]
    index2=search(database,hash_to_unpublish)
    del database["sha-1"][index2]                        #Updating local database.
    del database["filename"][index2]
    data = json.dumps(globaldata)
    for c in peersock:                                   #Sending updated global data to all peers.
        if c == ip_dc_machine:
            pass
        else:
            updateSock=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            updateSock.connect((c,9578))
            updateSock.sendall(str.encode(data))
            updateSock.close()
    return




#-----------------------------------------------------------------------------------------------------#
#Implementation of Network (p2p web)




#Function to peer to particular system by its name
def peering(getname):
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    serverAddress = socket.gethostbyname(getname)
    print('Connect to : ', serverAddress)
    global peersock
    peersock.append(serverAddress)
    client.connect((serverAddress, 9578))
    client.send(str.encode("I am your peer"))
    return




#For making server threads for each client connected
class serverWork:
    def __init__(self, clientAddress, clientSocket):
        self.socket = clientSocket
        print("New Connection from : ", clientAddress)
        self.clientAddress = clientAddress
        self.run()

    def run(self):
        while True:
            data = self.socket.recv(1024)
            data_received = data.decode("utf-8")
            if data_received == "I am your peer":                   #Updating peer table if peer is connected
                name=socket.gethostbyaddr(self.clientAddress[0])
                update(peers,"NAME OF PEER",name[0])
                update(peers,"IP ADDRESS",self.clientAddress[0])
                update(peers, "PORT", self.clientAddress[1])
            else:
                try:                                                #Updating globaldata if new update received from peer
                    new_update = json.loads(data_received)
                    global globaldata
                    globaldata = new_update
                    globaldata_to_send = json.dumps(globaldata)
                    for c in peersock:
                        if c == ip_dc_machine or c == self.clientAddress[0]:
                            pass
                        else:
                            try:                                    #Sending updated globaldata to all peers
                                updateSock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                                updateSock.connect((c, 9578))
                                updateSock.sendall(str.encode(globaldata_to_send))
                                updateSock.close()
                            except TimeoutError:                    #Handling Timeout error
                                print(" ")
                        return
                except ValueError:                                  #Handling Value Error
                    print (" ")
                    return




#Function to accept peering request acting as a server at port:9578
def serverThread(serverSock, host, port):
    print("Server Started listening at : ", host, " at port", port)
    print("Waiting.................")
    while True:
        serverSock.listen(5)
        clientSock, clientAddress = serverSock.accept()
        global peersock
        peersock.append(clientAddress[0])                               #Adding peer socket to list
        newThreadedclient = threading.Thread(target=serverWork, args=(clientAddress, clientSock))
        newThreadedclient.start()                                       #Starting new thread for each client connected
    return





#-----------------------------------------------------------------------------------------------------#
#Implementation of HTTP response



#Function for getting requested file from node by providing IP Address and Hash of the file
def connect_node(ip,hash_received):
    s = socket.socket()
    port =fix_fetch_port
    s.connect((ip, port))
    s.send(str.encode(hash_received))                   #Sending Hash received
    data=s.recv(1024)
    file=data.decode('utf-8')                           #Receiving name of the file requested
    with open(file, 'wb') as f:
        print('file opened')
        while True:
            print('receiving data...')                  #Receiving the file
            data = s.recv(16348)
            if not data:
                break
            f.write(data)

        f.close()
        print('Successfully got the file')
        s.close()                                       #Closing the connection with the node
        print('connection closed')
        return file




#Handling HTTP Request
class HTTPRequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):                                           #Handling GET command
        extracted_hash = self.path
        extracted_hash = extracted_hash[1:]                     #Extracting Hash from GET
        op=is_present(database,extracted_hash)                              #Checking if file is present in node's local database
        print("fetching")
        if op=="present":                                       #File is present in node's database
            f1 = filename(extracted_hash)
            n, d, content_type = str(f1).partition(".")         #Finding Content type of the file
            if (content_type == "txt" or content_type=="html"): #Serving html/txt request
                f = open(f1, 'rb')                              #Opening requested file
                self.send_response(200)                         #Sending 200 response
                self.send_header('content-type', 'text/html')
                self.end_headers()                              #Sending header first
                self.wfile.write(f.read())                      #Sending file to client
                f.close()
                self.connection.close()
                return True
            else:                                               #Serving image/png request
                f = open(f1)
                self.send_response(200)
                self.send_header('Content-type', 'image/png')
                self.end_headers()
                with open(f1, 'rb') as bindata:
                    shutil.copyfileobj(bindata, self.wfile)
                f.close()
                self.connection.close()
                return True

        else:                                                               #File is not present in node's local database
            index_in_globaltable=search(globaldata,extracted_hash)          #Finding index of file in globaldata
            if index_in_globaltable == -1 :
                self.send_error(404, 'file not found')                      #File not found in globaldata
                self.connection.close()
                return True

            else:
                ip_of_server = globaldata["ip"][index_in_globaltable]       #Finding IP of node who has the file
                print("connecting")
                file = connect_node(ip_of_server, extracted_hash)           #Connecting to that node
                f = open(file, 'r')
                p1, p2, content_type1 = str.partition(file, ".")
                if content_type1 == "png":                                  #Serving image/png request
                    self.send_response(200)
                    self.send_header('Content-type', 'image/png')
                    self.end_headers()
                    with open(file, 'rb') as bindata:
                        shutil.copyfileobj(bindata, self.wfile)
                    f.close()
                    self.connection.close()
                    return
                else:                                                       #Serving html/txt request
                    f = open(file, 'rb')
                    self.send_response(200)
                    self.send_header('content-type', 'text/html')
                    self.end_headers()
                    self.wfile.write(f.read())
                    f.close()
                    self.connection.close()
                    return




#Function to run HTTP server continously on port:1500
def httprun(add):
    httpser = http.server.HTTPServer(add, HTTPRequestHandler)
    print('HTTP Server running on', add[1])
    httpser.serve_forever()
    return




#Function to send file requested by node in p2pweb
#Listening on port:2100
def fetchserver():
    port = 2100
    s = socket.socket()
    host = socket.gethostbyname(socket.gethostname())
    s.bind((host, port))
    s.listen(5)
    while True:
        conn, addr = s.accept()
        hash_in_database = conn.recv(1024)                      #Receiving hash from node requesting the file
        index_of_hash=search(database,hash_in_database)
        file=database["filename"][index_of_hash]                #Finding Filename by searching Database
        conn.send(str.encode(file))                             #Sending filename to node requesting the file
        f = open(file, 'rb')
        l = f.read(1024)
        while (l):
            conn.send(l)
            l = f.read(1024)
        f.close()
        print('File sent to requesting Node')
        conn.close()                                            #Closing the connection after file is sent
    return


def commandInput():
    while True:
            userip = input("Which command do you want to execute ?")
            command = str(userip[0:2])
            if command == "PU":
                p1, p2, p3 = str(userip).partition("<")
                q1, q2, q3 = str(p3).partition(".")
                r1, r2, r3 = str(q3).partition(">")
                publish(q1, q2, r1)
            elif command == "SH":
                command_part = str(userip[5:7])
                if command_part == "ME":
                    show_metadata()
                elif command_part == "PU":
                    show_published()
                else:
                    show_peers()
            elif command == "PE":
                a1, a2, a3 = str(userip).partition("<")
                b1, b2, b3 = str(a3).partition(">")
                hostname = b1
                c1, c2, c3 = str(b3).partition("<")
                d1, d2, d3 = str(c3).partition(">")
                port = d1
                ip = socket.gethostbyname(hostname)
                update(peers, "NAME OF PEER", hostname)
                update(peers, "IP ADDRESS", ip)
                update(peers, "PORT", port)
                peering(hostname)
            else:
                a1, a2, a3 = str(userip).partition("<")
                b1, b2, b3 = str(a3).partition(">")
                hash_unpublish = b1
                unpublish(hash_unpublish)
    return




In [ ]:
!pip install flask

In [ ]:
cmdThread = threading.Thread(target=commandInput, args=())
cmdThread.start()                                               #Thread accepting user input i.e Commands
host =socket.gethostbyname(socket.gethostname())
port = 9578
global ip_dc_machine
ip_dc_machine = host
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((host, port))
newThreadedserver = threading.Thread(target=serverThread, args=(server, host, port))
newThreadedserver.start()                                       #Thread running Server for peering request on port:9578
add = (host, 1500)
httpthread = threading.Thread(target=httprun, args=(add,))
httpthread.start()                                              #Thread running HTTP Server on port:1500
fetchthread = threading.Thread(target=fetchserver())
fetchthread.start()                                             #Thread running Fetch Server on port:2100

hostname = "localhost"
port = 800

ip = socket.gethostbyname(hostname)
update(peers, "NAME OF PEER", hostname)
update(peers, "IP ADDRESS", ip)
update(peers, "PORT", port)
peering(hostname)

In [1]:
!pip install python-dotenv

In [ ]:
# Import Module
import ftplib

# Fill Required Information
HOSTNAME = "ftp.dlptest.com"
USERNAME = "dlpuser"
PASSWORD = "rNrKYTX9g7z3RgJRmxWuGHbeu"

In [ ]:
# Connect FTP Server
ftp_server = ftplib.FTP(HOSTNAME, USERNAME, PASSWORD)
# force UTF-8 encoding
ftp_server.encoding = "utf-8"

In [ ]:
# Enter File Name with Extension
filename = "test.txt"
# Read file in binary mode
with open(filename, "rb") as file:
# Command for Uploading the file "STOR filename"
    ftp_server.storbinary(f"STOR {filename}", file)

In [ ]:
ftp_server.dir()

In [ ]:
# Enter File Name with Extension
filename = "COORD-00021_data_parameters_OPHELIA-02000_1669848000.csv"

# Write file in binary mode
with open(filename, "wb") as file:
# Command for Downloading the file "RETR filename"
    ftp_server.retrbinary(f"RETR {filename}", file.write)

In [ ]:
# Display the content of downloaded file
file= open(filename, "r")
print('File Content:', file.read())

# Close the Connection
ftp_server.quit()

In [ ]:
from flask import Flask

app = Flask(__name__)


@app.route('/hello/', methods=['GET', 'POST'])
def welcome():
    return "Hello World!"
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=105)